In [ ]:
import numpy as np
import pandas as pd
import json
import os
import openai
import random

#API Key
openai.api_key = '' #deleted for uploading to github


#reading in data
df = pd.read_csv("/Users/stevenslater/Desktop/FinalProject/Data/trainwindowsimproved.csv")
df_val = pd.read_csv("/Users/stevenslater/Desktop/FinalProject/Data/valwindowsimproved.csv")
df_test = pd.read_csv("/Users/stevenslater/Desktop/FinalProject/Data/testwindowsimproved.csv")
df.head()


In [32]:
#ChatGPT API function - with pricing.
def ask_gpt(question):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4-1106-preview",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": question}
            ]
        )

        answer = response['choices'][0]['message']['content']
        tokens_used = response['usage']['total_tokens']
        input_tokens = len(question.split())  # approx count of input tokens
        return answer, tokens_used, input_tokens

    except openai.error.OpenAIError as e:
        return f"Error: {str(e)}", 0, 0
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}", 0, 0

In [4]:
#Template Construction
def get_template(example):
    
    metaPrompt = """Our definition of vulnerability refers to customers who, due to their personal circumstances, are especially susceptible to harm. All customers are at risk of becoming vulnerable and this risk is increased by characteristics of vulnerability related to 4 key drivers.
    Health - health conditions or illnesses that affect ability to carry out day-to-day tasks.

    Life events - life events such as bereavement, job loss or relationship breakdown.

    Resilience - low ability to withstand financial or emotional shocks.

    Capability - low knowledge of financial matters or low confidence in managing money (financial capability). Low capability in other relevant areas such as literacy, or digital skills
    """

    conversation = "\nConversation to classify: " + example + "\n"

    instruction = "\nFrom the above conversation, is the customer vulnerable based on our definition of vulnerability, only answer Yes or No, and you must say only Yes or No."

    
    prompt = metaPrompt + conversation + instruction

    return prompt

In [34]:
def query_api(sampled_df):
    
    results = []
    total_input_tokens = 0
    total_output_tokens = 0

    
    for _, row in sampled_df.iterrows():
        
        
        utterance = row['conversation_chunks']
        #get prompt for that conversation
        prompt = get_template(utterance)
        
        #use GPT-4 to get the answer
        answer, tokens_used, input_tokens = ask_gpt(prompt)
        total_input_tokens += input_tokens
        total_output_tokens += tokens_used - input_tokens

        
        answer = answer.rstrip('.').strip()

        # convert the answer to a binary value
        answer_binary = 1 if answer.lower() == "yes" else 0
        
        
        results.append({
            'Data': utterance,
            'Actual Label': row['labels'],
            'ChatGPT Label': answer_binary
        })
    
    #monitoring cost
    cost_per_1k_input_tokens = 0.01
    cost_per_1k_output_tokens = 0.03
    total_cost = (total_input_tokens / 1000 * cost_per_1k_input_tokens) + \
                 (total_output_tokens / 1000 * cost_per_1k_output_tokens)

    print(f"Total Cost: {total_cost}")
    
    # convert the results to DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df


In [ ]:
#TESTING
df_test = pd.read_csv("/Users/stevenslater/Desktop/FinalProject/Data/testwindowsimproved.csv")
#df_test = pd.concat([df_test.drop(['data'], axis=1), df_test['data'].apply(pd.Series)], axis=1)
df_test.head()

In [ ]:
results_test_experiment_1 =  results_test
results_test_experiment_1.to_csv('results_test_experiment_1.csv')
results_test_experiment_1

In [ ]:
results_test_experiment_1 =  pd.read_csv('results_test_experiment_1.csv')
results_test_experiment_1

In [ ]:
#METRICS 
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

actual_labels = results_test_experiment_1['Actual Label'].astype(int)  # Convert to numeric
predicted_labels = results_test_experiment_1['ChatGPT Label'].astype(int)  # Convert to numeric

# Calculate metrics
accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels)
recall = recall_score(actual_labels, predicted_labels)
f1 = f1_score(actual_labels, predicted_labels)

# create a confusion matrix
confusion = confusion_matrix(actual_labels, predicted_labels)

# print the metrics and confusion matrix
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(confusion)
